In [ ]:
# Let's build GPT: from scratch, in code, spelled out.(https://youtu.be/kCc8FmEb1nY?si=YTkDSOYk5S_CJVtz)

In [3]:
with open('tinyshakespeare.txt', 'r') as f:
    text = f.read()

In [9]:
print(f'sample: \n{text[:100]}\n\ntotal text lenght : {len(text)}')

sample: 
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

total text lenght : 1115394


In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [13]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode =lambda s: [stoi[c] for c in s]
decode =lambda l: ''.join([itos[i] for i in l])
print(encode('hi there!'))
print(decode(encode('hi there!')))

[46, 47, 1, 58, 46, 43, 56, 43, 2]
hi there!


In [14]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(f'sample : \n{data[:100]}\n\nshape = {data.shape}, dtype = {data.dtype}')

sample : 
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

shape = torch.Size([1115394]), dtype = torch.int64


In [15]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [17]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context, '--->', target)

tensor([18]) ---> tensor(47)
tensor([18, 47]) ---> tensor(56)
tensor([18, 47, 56]) ---> tensor(57)
tensor([18, 47, 56, 57]) ---> tensor(58)
tensor([18, 47, 56, 57, 58]) ---> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) ---> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) ---> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) ---> tensor(58)


In [19]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = {
        'train' : train_data,
        'val' : val_data
    }[split]
    ix = torch.randint(len(data) - block_size, (block_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print(f'input :\n{xb.shape}\n{xb}\ntarget :\n{yb.shape}\n{yb}')

input :
torch.Size([8, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])
target :
torch.Size([8, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39],
        [43, 60, 43, 52,  1, 63, 43, 39],
        [43, 42,  8,  0, 25, 63,  1, 45],
        [42,  5, 57,  1, 57, 39, 49, 43],
        [57, 58, 63,  6,  1, 58, 46, 47]])
